In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from collections import Counter
import plotly.graph_objects as go


import datetime as dt
import time
from tqdm import tqdm
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import re

import warnings
warnings.filterwarnings('ignore')

from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

from yellowbrick.cluster.elbow import kelbow_visualizer

from itertools import combinations

# 정규성 검정 모듈
from scipy.stats import shapiro

# 등분산 검정 모듈
from scipy.stats import bartlett
from scipy.stats import fligner
from scipy.stats import levene


from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu , ranksums

In [10]:
# pip install ipykernel
# pip install --upgrade nbformat

In [11]:
df = pd.read_csv('result.csv')
df

,고객ID,Recency,Frequency,Monetary,Length,평균구매주기,구매카테고리수,총납세액,총구매수량,총할인금액,...,성별,used개수,not used개수,Used_ratio_per_F,Clicked_ratio_per_F,second_category_pca_1,second_category_pca_2,third_category_pca_1,third_category_pca_2,third_category_pca_3
0,USER_0000,-0.365961,0.693147,4.628801,0.000000,-0.475194,-1.271454,1.740150,1.386294,2.331853,...,0,1.098612,0.000000,0.693147,0.000000,0.232740,0.886966,-0.293191,-0.537568,0.691002
1,USER_0001,-0.837001,3.465736,9.642689,5.411646,0.196981,1.238119,7.634776,5.837730,6.662555,...,1,3.044522,2.708050,0.010695,0.013882,0.722953,-0.421938,0.752657,0.120987,0.028289
2,USER_0002,-0.699614,2.197225,7.375936,4.787492,0.053730,-0.016668,5.321061,5.347108,4.880565,...,1,2.484907,0.693147,0.058064,0.058064,0.142694,0.228191,-0.261827,-0.365882,-0.677775
3,USER_0003,-1.249160,2.484907,7.272056,0.000000,-1.466928,-0.644061,5.005718,3.091042,5.031855,...,1,1.945910,1.386294,0.031582,0.041891,-0.668210,-0.171614,-0.532305,0.707652,0.080600
4,USER_0004,-0.365961,2.639057,7.341466,0.000000,-0.475194,0.297029,5.028867,4.043051,5.336326,...,0,2.484907,2.197225,0.023232,0.035681,0.232740,0.886966,-0.532305,0.707652,0.080600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,USER_1463,1.233611,1.386294,6.521681,0.000000,1.320945,-0.644061,3.955469,3.850148,2.500616,...,0,1.386294,0.693147,0.133531,0.133531,0.232740,0.886966,-0.261827,-0.365882,-0.677775
1464,USER_1464,-0.562227,2.995732,7.875863,4.663439,-0.563348,1.551815,5.628235,4.955827,5.443767,...,1,2.833213,1.386294,0.020834,0.027257,-0.668210,-0.171614,0.752657,0.120987,0.028289
1465,USER_1465,0.487799,1.098612,4.863862,4.343805,0.252077,-1.271454,2.945827,2.397895,1.517980,...,1,1.386294,0.693147,0.171850,0.223144,0.232740,0.886966,-0.293191,-0.537568,0.691002
1466,USER_1466,-0.738867,0.693147,5.813533,0.000000,-0.893926,-1.585151,3.427515,1.098612,0.000000,...,0,0.000000,0.693147,0.000000,0.000000,0.157450,0.278253,-0.293191,-0.537568,0.691002


In [12]:
core = df.iloc[:, 1:]
core

,Recency,Frequency,Monetary,Length,평균구매주기,구매카테고리수,총납세액,총구매수량,총할인금액,총배송료,...,성별,used개수,not used개수,Used_ratio_per_F,Clicked_ratio_per_F,second_category_pca_1,second_category_pca_2,third_category_pca_1,third_category_pca_2,third_category_pca_3
0,-0.365961,0.693147,4.628801,0.000000,-0.475194,-1.271454,1.740150,1.386294,2.331853,4.330733,...,0,1.098612,0.000000,0.693147,0.000000,0.232740,0.886966,-0.293191,-0.537568,0.691002
1,-0.837001,3.465736,9.642689,5.411646,0.196981,1.238119,7.634776,5.837730,6.662555,5.661084,...,1,3.044522,2.708050,0.010695,0.013882,0.722953,-0.421938,0.752657,0.120987,0.028289
2,-0.699614,2.197225,7.375936,4.787492,0.053730,-0.016668,5.321061,5.347108,4.880565,4.407573,...,1,2.484907,0.693147,0.058064,0.058064,0.142694,0.228191,-0.261827,-0.365882,-0.677775
3,-1.249160,2.484907,7.272056,0.000000,-1.466928,-0.644061,5.005718,3.091042,5.031855,4.423528,...,1,1.945910,1.386294,0.031582,0.041891,-0.668210,-0.171614,-0.532305,0.707652,0.080600
4,-0.365961,2.639057,7.341466,0.000000,-0.475194,0.297029,5.028867,4.043051,5.336326,5.046581,...,0,2.484907,2.197225,0.023232,0.035681,0.232740,0.886966,-0.532305,0.707652,0.080600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,1.233611,1.386294,6.521681,0.000000,1.320945,-0.644061,3.955469,3.850148,2.500616,4.557554,...,0,1.386294,0.693147,0.133531,0.133531,0.232740,0.886966,-0.261827,-0.365882,-0.677775
1464,-0.562227,2.995732,7.875863,4.663439,-0.563348,1.551815,5.628235,4.955827,5.443767,5.407396,...,1,2.833213,1.386294,0.020834,0.027257,-0.668210,-0.171614,0.752657,0.120987,0.028289
1465,0.487799,1.098612,4.863862,4.343805,0.252077,-1.271454,2.945827,2.397895,1.517980,2.602690,...,1,1.386294,0.693147,0.171850,0.223144,0.232740,0.886966,-0.293191,-0.537568,0.691002
1466,-0.738867,0.693147,5.813533,0.000000,-0.893926,-1.585151,3.427515,1.098612,0.000000,1.945910,...,0,0.000000,0.693147,0.000000,0.000000,0.157450,0.278253,-0.293191,-0.537568,0.691002


In [13]:
pca = PCA(n_components = 2)

rfm_done_pca = pd.DataFrame(data=pca.fit_transform(core), columns=['pca1', 'pca2'])

for method in ['random', 'k-means++']:
    print(method)
    for k in range(3, 10):
        kmeans = KMeans(n_clusters=k, random_state=42, init='k-means++').fit(rfm_done_pca)
        silhouette_avg = silhouette_score(rfm_done_pca, kmeans.labels_)
        print("k = {} -> silhouette index {}".format(k, silhouette_avg))
        core[f'k_{k}_labels'] = kmeans.labels_
    
    print()

random
k = 3 -> silhouette index 0.45573803035468735
k = 4 -> silhouette index 0.46888878256389055
k = 5 -> silhouette index 0.502266117683849
k = 6 -> silhouette index 0.49576921510767835
k = 7 -> silhouette index 0.4676701867444823
k = 8 -> silhouette index 0.46893453203181135
k = 9 -> silhouette index 0.44490912300674657

k-means++
k = 3 -> silhouette index 0.45573803035468735
k = 4 -> silhouette index 0.46888878256389055
k = 5 -> silhouette index 0.502266117683849
k = 6 -> silhouette index 0.49576921510767835
k = 7 -> silhouette index 0.4676701867444823
k = 8 -> silhouette index 0.46893453203181135
k = 9 -> silhouette index 0.44490912300674657



In [14]:
cols1 = ['Recency', 'Frequency', 'Monetary', 'Length', '평균구매주기', '구매카테고리수']
cols2 = ['총납세액', '총구매수량', '총할인금액', '총배송료', '최다구매월구매횟수']
cols3 = ['고유거래일수', '성별', 'used개수','not used개수', 'Used_ratio_per_F', 'Clicked_ratio_per_F']
cols4 = ['second_category_pca_1', 'second_category_pca_2',
       'third_category_pca_1', 'third_category_pca_2', 'third_category_pca_3']


cols_groups = [cols1, cols2, cols3, cols4]

def plot_radar_from_centroid(df_centroids, cols):
  df_centroids = df_centroids[cols]
  fig = go.Figure()
  categories = df_centroids.columns
  for row in df_centroids.iterrows():
    fig.add_trace(go.Scatterpolar(
        r=row[1].tolist(),
        theta=categories,
        fill='toself',
        name='cluster {}'.format(row[0])
    ))

  fig.update_layout(
      autosize=False,
      width=1000,
      height=800,
  )
  fig.show()

In [15]:
for cols in cols_groups:
    for k in range(3, 10):
        kmeans = KMeans(n_clusters=k, random_state=42, init='random').fit(core.iloc[:, :23])
        c = Counter()
        c.update(kmeans.labels_)

        core[f'k_{k}_labels'] = kmeans.labels_

        df_centroids = pd.DataFrame(kmeans.cluster_centers_, columns=core.iloc[:, :23].columns)
        label_counts = [c[x] for x in df_centroids.index]
        df_centroids['size'] = label_counts

        plot_radar_from_centroid(df_centroids, cols)

In [16]:
cols1 = ['Recency', 'Frequency', 'Monetary', 'Length']

cols2 = ['평균구매주기', '구매카테고리수','총납세액', '총구매수량', 
         '총할인금액', '총배송료', '최다구매월구매횟수', '고유거래일수']
cols3 = ['성별', 'used개수','not used개수', 
        'Used_ratio_per_F', 'Clicked_ratio_per_F']
cols4 = ['second_category_pca_1','second_category_pca_2',
         'third_category_pca_1', 'third_category_pca_2', 'third_category_pca_3']



elements = [3, 4, 5, 6, 7, 8, 9]



for cols in [cols1, cols2, cols3, cols4]:
    values = []
    for col in cols:  
        for k in elements:
            results = []
            # print(col)
            # print(f'k={k}')
            combination = list(combinations([_ for _ in range(k)], 2))
            total_combis = len(combination)
            for combis in combinations([_ for _ in range(k)], 2):
                group1 = core[core[f'k_{k}_labels'] == combis[0]][col]
                group2 = core[core[f'k_{k}_labels'] == combis[1]][col]
                
                # print(f"정규성 검정 - {combis}")
                # print(f'정규성 검정 - 클러스터 {combis[0]} -  p_value가 0.05보다 작은가 : {shapiro(group1)[1] < 0.05}')
                # print(f'{combis[0]} : {shapiro(group1)}')
                
                # print()
                # print(f'정규성 검정 - 클러스터 {combis[1]} -  p_value가 0.05보다 작은가 : {shapiro(group2)[1] < 0.05}')
                # print(f'{combis[1]} : {shapiro(group2)}')
                result1 = shapiro(group1)[1] < 0.05
                result2 = shapiro(group2)[1] < 0.05
                result_shapiro = (result1 * result2) > 0
                # print()
                
                # print(f'등분산 검정 - {combis}')
                # print('등분산 검정 - bartlett - p_value가 0.05보다 작은가 : ', bartlett(group1,group2)[1] < 0.05)
                # print(bartlett(group1,group2))
                
                # print()
                # print('등분산 검정 - fligner(median) - p_value가 0.05보다 작은가 : ', fligner(group1,group2,center='median')[1] < 0.05)
                # print(fligner(group1,group2,center='median')) #default

                # print('등분산 검정 - fligner(mean) - p_value가 0.05보다 작은가 : ', fligner(group1,group2,center='mean')[1] < 0.05)
                # print(fligner(group1,group2,center='mean'))
                
                # print()
                # print('등분산 검정 - levene(median) - p_value가 0.05보다 작은가 : ',levene(group1,group2, center='median')[1] < 0.05)
                # print(levene(group1,group2, center='median')) #default
                
                
                # print('등분산 검정 - levene(mean) - p_value가 0.05보다 작은가 : ',levene(group1,group2, center='mean')[1] < 0.05)
                # print(levene(group1,group2,center='mean'))
                
                
                result3 = bartlett(group1,group2)[1] < 0.05
                result4 = fligner(group1,group2,center='median')[1] < 0.05
                result5 = fligner(group1,group2,center='mean')[1] < 0.05
                result6 = levene(group1,group2, center='median')[1] < 0.05
                result7 = levene(group1,group2, center='mean')[1] < 0.05
                
                result_homoskedasticity = sum([result3, result4, result5, result6, result7]) >= 3
                
                # print()
                # print(f'정규성 검정 결과 : {result_shapiro}')
                # print(f'등분산 검정 결과 : {result_homoskedasticity}')
                # print()
                
                if (result_shapiro == True) and (result_homoskedasticity == True):
                    # print('등분산 t-test실시')
                    ttest_true = ttest_ind(group1,group2,equal_var=True)
                    # print('등분산 t-test_p_value가 0.05보다 작은가 : ',ttest_true[1] < 0.05)
                    # print(ttest_true)
                    
                    result = ttest_true
                
                elif (result_shapiro == True) and (result_homoskedasticity == False):
                    # print('이분산 t-test실시')
                    ttest_false = ttest_ind(group1,group2,equal_var=False)
                    # print('이분산 t-test_p_value가 0.05보다 작은가 : ',ttest_false[1] < 0.05)
                    # print(ttest_false)
                    
                    result = ttest_false

                
                else:
                    # print('비모수검정 실시')

                    test_ranks = ranksums(group1,group2)

                    # print('ranksums검정_p_value가 0.05보다 작은가 : ',test_ranks[1] < 0.05)
                    # print(test_ranks)                
                    
                    result = test_ranks
                    
                
                results.append(result[1] < 0.05)
                    
            # print(f'{sum(results)}/{total_combis}')
            values.append(f'{sum(results)}/{total_combis}')
            
    temp_values = np.array(values).reshape(len(cols), -1)
    result_table = pd.DataFrame(data=temp_values, columns=elements, index=cols)
    display(result_table)
        #     print("+++++++++++++++++++++++++++++++++++")
        # print('========================================================================')


,3,4,5,6,7,8,9
Recency,2/3,5/6,8/10,12/15,18/21,24/28,31/36
Frequency,3/3,6/6,10/10,15/15,21/21,28/28,35/36
Monetary,3/3,6/6,10/10,15/15,21/21,28/28,33/36
Length,3/3,5/6,9/10,13/15,18/21,24/28,32/36


,3,4,5,6,7,8,9
평균구매주기,3/3,5/6,10/10,11/15,16/21,20/28,29/36
구매카테고리수,3/3,6/6,10/10,15/15,21/21,28/28,33/36
총납세액,3/3,6/6,10/10,15/15,21/21,28/28,33/36
총구매수량,3/3,6/6,10/10,15/15,21/21,28/28,33/36
총할인금액,3/3,6/6,10/10,14/15,21/21,28/28,35/36
총배송료,3/3,6/6,10/10,15/15,21/21,28/28,34/36
최다구매월구매횟수,3/3,6/6,9/10,15/15,21/21,27/28,36/36
고유거래일수,2/3,6/6,9/10,12/15,20/21,26/28,33/36


,3,4,5,6,7,8,9
성별,0/3,1/6,2/10,2/15,1/21,1/28,2/36
used개수,3/3,6/6,10/10,15/15,21/21,28/28,33/36
not used개수,3/3,6/6,10/10,15/15,21/21,28/28,32/36
Used_ratio_per_F,3/3,5/6,10/10,15/15,20/21,27/28,33/36
Clicked_ratio_per_F,3/3,6/6,10/10,15/15,21/21,28/28,34/36


,3,4,5,6,7,8,9
second_category_pca_1,3/3,5/6,8/10,11/15,13/21,18/28,23/36
second_category_pca_2,3/3,5/6,7/10,10/15,15/21,19/28,25/36
third_category_pca_1,3/3,5/6,9/10,14/15,16/21,23/28,29/36
third_category_pca_2,2/3,4/6,8/10,13/15,17/21,21/28,27/36
third_category_pca_3,2/3,5/6,6/10,12/15,16/21,20/28,25/36
